In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from pasna_analysis import DataLoader, Experiment

experiment_name = '20240919_25C'
exp_path = Path.cwd().parent.joinpath('data',experiment_name)
exp_data = DataLoader(exp_path)
exp = Experiment(DataLoader(exp_path), to_exclude=[16])

Visualize developmental time for a single embryo.

In [ ]:
i = 3

emb = exp.embryos[i]
W = emb.developmental_time()
X = emb.vnc_length[:, 0] / 60

np_lin_fit_values = emb.lin_developmental_time()

fig, ax = plt.subplots()
ax.plot(X, W, label="raw")
ax.plot(X, np_lin_fit_values, label="linear fit")
ax.set_title(f"VNC length - {emb.name}")
ax.set_xlabel("t(min)")
ax.set_ylabel("Developmental time")
ax.legend()

fig.tight_layout()

Visualize developmental time, for all embryos of the current experiment.

In [ ]:
fig, (ax, ax2) = plt.subplots(2, figsize=(10, 12))

for emb in exp.embryos:
    Y = emb.vnc_length_filtered
    W = emb.developmental_time()
    X = emb.vnc_length[:, 0] / 60

    ax.plot(X, Y, label=emb.name)
    ax2.plot(X, W)


ax.set_title(f"VNC length - {exp_data.name}")
ax.set_xlabel("t(min)")
ax.set_ylabel("um")
ax.legend()

fig.tight_layout()

Try to find the best fit for developmental time. 
Tried to use a linear regression and a log regression.
Log regression seems to perform a bit better.

In [ ]:
from scipy.optimize import curve_fit


def average(arr):
    return sum(arr) / len(arr)


def linear_func(x, a, b):
    return a * x + b


def log_func(x, a, b, c):
    return a * np.log(x + b) + c


def fit_data(x, y, func, p0=None):
    params, _ = curve_fit(func, x, y, p0=p0, maxfev=1_000_000)
    y_fit = func(x, *params)

    ss_res_lin = np.sum((y - y_fit) ** 2)
    ss_tot_lin = np.sum((y - np.mean(y)) ** 2)
    r2 = 1 - (ss_res_lin / ss_tot_lin)

    return params, y_fit, r2


def linearize(x, a, b, c):
    return a * np.log(x) + a * np.log(b) + c


fig, ax = plt.subplots(2, figsize=(10, 12))
r2_lins = []
r2_logs = []
for emb in exp.embryos:
    x = emb.vnc_length[:, 0] / 60
    dev_time = emb.size / emb.vnc_length[:, 1]

    params, y_fit_lin, r2_lin = fit_data(x, dev_time, linear_func)
    params, y_fit_log, r2_log = fit_data(x, dev_time, log_func, p0=[2, 2, 2])
    r2_lins.append(r2_lin)
    r2_logs.append(r2_log)
    ax[0].plot(x, y_fit_lin)
    ax[1].plot(x, y_fit_log)


print(average(r2_lins), len(exp.embryos))
print(average(r2_logs), len(exp.embryos))

fig.supxlabel("t(min)")
fig.supylabel("developmental time")
ax[0].set_title("Linear fit")
ax[1].set_title("Log fit")
fig.tight_layout()